#  Titanic Dataset  with Neural Network

Serajus Salehin, ID : 2017-3-60-018
Zubayar Mahatab Md Sakif, ID :2018-1-60-105
Israt Jahan Mridula, ID : 2017-1-60-108
Md. Nazmul islam,ID : 2017-2-60-38 



In [529]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

# Neural Network
#from tensorflow import keras
import keras

from keras.models import Sequential 
from keras.layers import Dense

# load the data
df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')
df = df_train.append(df_test , ignore_index = True)

print(df_train.shape, df_test.shape,df.shape)

((891, 12), (418, 11), (1309, 12))


### Pclass Analysis

In [530]:
df['Pclass'].isnull().sum()

0

In [531]:
df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean()

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


### Name To Title

In [532]:
df.Name.head(10)

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
5                                     Moran, Mr. James
6                              McCarthy, Mr. Timothy J
7                       Palsson, Master. Gosta Leonard
8    Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)
9                  Nasser, Mrs. Nicholas (Adele Achem)
Name: Name, dtype: object

In [533]:
df['Title'] = df.Name.map( lambda x: x.split(',')[1].split( '.' )[0].strip())

df['Title'].value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Major             2
Mlle              2
Ms                2
Mme               1
the Countess      1
Don               1
Lady              1
Sir               1
Jonkheer          1
Dona              1
Capt              1
Name: Title, dtype: int64

In [534]:
df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace(['Mme','Lady','Ms'], 'Mrs')
df.Title.loc[ (df.Title !=  'Master') & (df.Title !=  'Mr') & (df.Title !=  'Miss') 
             & (df.Title !=  'Mrs')] = 'Others'

df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.701087
2,Mr,0.156673
3,Mrs,0.796875
4,Others,0.318182


In [535]:
df['Title'].value_counts()

Mr        757
Miss      262
Mrs       201
Master     61
Others     28
Name: Title, dtype: int64

In [536]:
df = pd.concat([df, pd.get_dummies(df['Title'])], axis=1).drop(labels=['Name'], axis=1)

### Sex(Male:0 Female:1)

In [537]:
df.Sex.isnull().sum()

0

In [538]:
df[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean()


,Sex,Survived
0,female,0.742038
1,male,0.188908


In [539]:
df.Sex = df.Sex.map({'male':0, 'female':1})













### Cabin(Dropped)

In [540]:
df.Cabin.isnull().sum(axis=0)

1014

In [541]:
df = df.drop(labels=['Cabin'], axis=1)

### Embarked(Dropped)

In [542]:
df.Embarked.isnull().sum(axis=0)

2

In [543]:
df['Embarked'].value_counts()

S    914
C    270
Q    123
Name: Embarked, dtype: int64

In [544]:
df.Embarked.fillna('S' , inplace=True )

In [545]:
df[['Embarked', 'Survived','Pclass', 'Age']].groupby(['Embarked'], as_index=False).mean()

,Embarked,Survived,Pclass,Age
0,C,0.553571,1.851852,32.332170
1,Q,0.389610,2.894309,28.630000
2,S,0.339009,2.344978,29.298151


In [546]:
df = df.drop(labels='Embarked', axis=1)

### Age(RandomForestRegressor)

In [547]:
df.Age.isnull().sum()

263

In [548]:
df[['Title', 'Age']].groupby(['Title']).mean()

,Age
Title,
Master,5.482642
Miss,21.795236
Mr,32.252151
Mrs,36.930636
Others,45.074074


In [549]:
df[['Title', 'Age']].groupby(['Title']).std()

,Age
Title,
Master,4.161554
Miss,12.192794
Mr,12.422089
Mrs,12.872625
Others,11.303253


In [550]:
df_sub = df[['Age','Master','Miss','Mr','Mrs','Others','SibSp']]

X_train  = df_sub.dropna().drop('Age', axis=1)
y_train  = df['Age'].dropna()
X_test = df_sub.loc[np.isnan(df.Age)].drop('Age', axis=1)

regressor = RandomForestRegressor(n_estimators = 300)
regressor.fit(X_train, y_train)
y_pred = np.round(regressor.predict(X_test),1)
df.Age.loc[df.Age.isnull()] = y_pred
df.Age.isnull().sum(axis=0) 

0

### Ticket Analysis

In [551]:
df.Ticket.isnull().sum()

0

In [552]:
df.Ticket.head(20)

0            A/5 21171
1             PC 17599
2     STON/O2. 3101282
3               113803
4               373450
5               330877
6                17463
7               349909
8               347742
9               237736
10             PP 9549
11              113783
12           A/5. 2151
13              347082
14              350406
15              248706
16              382652
17              244373
18              345763
19                2649
Name: Ticket, dtype: object

In [553]:
df.Ticket = df.Ticket.map(lambda x: x[0])
df[['Ticket', 'Survived']].groupby(['Ticket'], as_index=False).mean()

,Ticket,Survived
0,1,0.630137
1,2,0.464481
2,3,0.239203
3,4,0.200000
4,5,0.000000
5,6,0.166667
6,7,0.111111
7,8,0.000000
8,9,1.000000
9,A,0.068966


In [554]:
df['Ticket'].value_counts()

3    429
2    278
1    210
S     98
P     98
C     77
A     42
W     19
7     13
F     13
4     11
6      9
L      5
5      3
9      2
8      2
Name: Ticket, dtype: int64

In [555]:
df['Ticket'] = df['Ticket'].replace(['A','W','F','L','5','6','7','8','9'], 'others')

df[['Ticket', 'Survived']].groupby(['Ticket'], as_index=False).mean()

,Ticket,Survived
0,1,0.630137
1,2,0.464481
2,3,0.239203
3,4,0.200000
4,C,0.340426
5,P,0.646154
6,S,0.323077
7,others,0.162162


In [556]:
df = pd.get_dummies(df,columns=['Ticket'])

## Trainning and Testing


In [557]:
#from sklearn.model_selection import train_test_split
#df = df.drop(labels=['SibSp','Parch','Fare','Title','PassengerId'], axis=1)
#y_true=df["Survived"]
#X_train,X_test,y_train,y_test = train_test_split(df,y_true,test_size=0.2,random_state=0)

In [558]:
df = df.drop(labels=['SibSp','Parch','Fare','Title','PassengerId'], axis=1)
y_train = df[0:891]['Survived'].values
X_train = df[0:891].drop(['Survived'], axis=1).values
y_test  = df[891:].drop(['Survived'], axis=1).values

In [559]:
#from sklearn.metrics import accuracy_score
#from sklearn.metrics import confusion_matrix

#def Model(model,name):
#    model.fit(X_train,y_train)
#    score = model.score(X_test, y_test)
#    model_train_score = model.score(X_train, y_train)
#    model_test_score = model.score(X_test, y_test)
#    prediction = model.predict(X_test)
#    print('{} Trainng Score {}\n'.format(name,model_train_score))
#    print('{} Testing Score {}\n'.format(name,model_test_score))
#    print('{} Testing Score {}\n'.format(name,score))

In [560]:
#from sklearn import svm
#clf = svm.SVC()
#Model(clf,"SVM")

In [561]:

#from sklearn.linear_model import LogisticRegression
#lr=LogisticRegression()
#Model(lr,"Logistic Regression")

In [562]:
from keras.initializers import glorot_uniform
model = Sequential()
model.add(Dense(9, activation = 'relu', input_dim = 16))
model.add(Dense(9,  activation = 'relu'))
model.add(Dense(5,  activation = 'relu'))
model.add(Dense(1,  activation = 'sigmoid'))

In [563]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

model.fit(X_train, y_train, batch_size = 32 , epochs = 500)

Epoch 1/500
891/891 [==============================] - 0s 136us/step - loss: 0.3476 - accuracy: 0.6139
Epoch 2/500
891/891 [==============================] - 0s 42us/step - loss: 0.2688 - accuracy: 0.6285
Epoch 3/500
891/891 [==============================] - 0s 40us/step - loss: 0.2297 - accuracy: 0.6319
Epoch 4/500
891/891 [==============================] - 0s 42us/step - loss: 0.2176 - accuracy: 0.6756
Epoch 5/500
891/891 [==============================] - 0s 41us/step - loss: 0.2018 - accuracy: 0.6880
Epoch 6/500
891/891 [==============================] - 0s 45us/step - loss: 0.1883 - accuracy: 0.7520
Epoch 7/500
891/891 [==============================] - 0s 40us/step - loss: 0.1763 - accuracy: 0.7475
Epoch 8/500
891/891 [==============================] - 0s 40us/step - loss: 0.1687 - accuracy: 0.7497
Epoch 9/500
891/891 [==============================] - 0s 42us/step - loss: 0.1556 - accuracy: 0.7811
Epoch 10/500
891/891 [==============================] - 0s 44us/step - loss: 0.14

##Prediction

In [564]:
#y_pred = model.predict(X_test,batch_size=32)
#y_final = (y_pred > 0.5).astype(int).reshape(X_test.shape[0])

#predicted_table = pd.DataFrame({'PassengerId': df_test['PassengerId'],'Age': df_test['Age'],'Sex': df_test['Sex'], 'Survived': y_final})
#predicted_table.to_csv('/content/predicted.csv', index=False)